<a href="https://colab.research.google.com/github/ARJUN108-verma/LangChain-/blob/main/34_Generation_stuffing_documents_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generation: stuffing documnets

In [ ]:
!pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
!pip install langchain_openai

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [ ]:
vectorstore = Chroma(persist_directory = "./intro-to-ds-lectures",
                     embedding_function = OpenAIEmbeddings(model='text-embedding-ada-002'))

In [ ]:
len(vectorstore.get()['documents'])

In [ ]:
retriever = vectorstore.as_retriever(search_type = 'mmr',
                                     search_kwargs = {'k':3,
                                                      'lambda_mult':0.7})

In [ ]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs = {'seed':365},
                  max_tokens = 250)

In [ ]:
question = "What software do data scientists use?"

In [ ]:
chain = {'context': retriever,
         'question': RunnablePassthrough()} | prompt_template

In [ ]:
chain.invoke(question)

In [ ]:
print("\nAnswer the following question:\nWhat software do data scientists use?\n\nTo answer the question, use only the following context:\n[Document(page_content='As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='It’s actually a software framework which was designed to address the complexity of big data and its computational intensity. Most notably, Hadoop distributes the computational tasks on multiple computers which is basically the way to handle big data nowadays. Power BI, SaS, Qlik, and especially Tableau are top-notch examples of software designed for business intelligence visualizations', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}), Document(page_content='Great! We hope we gave you a good idea about the level of applicability of the most frequently used programming and software tools in the field of data science. Thank you for watching!', metadata={'Course Title': 'Introduction to Data and Data Science', 'Lecture Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})]\n\nAt the end of the response, specify the name of the lecture this context is taken from in the format:\nResources: *Lecture Title*\nwhere *Lecture Title* should be substituted with the title of all resource lectures.\n")